# Python 2

In [1]:
import numpy as np

from LHCO_reader import LHCO_reader

import math
import os

# Plotting library
import matplotlib.pyplot as plt

import pandas as pd

In [3]:
!ls ../data/Signal/lhco/*.lhco

../data/Signal/lhco/BP0.lhco	    ../data/Signal/lhco/eventsBP0-sinMET.lhco
../data/Signal/lhco/BP1.lhco	    ../data/Signal/lhco/events_BP1-sinMET.lhco
../data/Signal/lhco/eventsBP0.lhco


## Signal

In [4]:
Folder    = '../data/Signal/lhco/'
datFolder = '../data/Signal/'


signal_photon  = []
signal_jet     = []
signal_lepton  = []
signal_MET     = []
signal_hadinfo = []

signal_runs = 2
for ii in range(0, signal_runs):
    print(ii)
    if ii not in [0]: # BPs que ya fueron analizados
        inputevents = LHCO_reader.Events(f_name = Folder + "events_BP" + str(ii) + "-sinMET.lhco")

        signal_init_ev = len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')

        signal_cut_ev  = 0
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -5.0 < inputevents[i]["jet"][0]["eta"] < 5.0:

                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.5 < inputevents[i]["photon"][0]["eta"] < 2.5:
                        if inputevents[i]["MET"][0]["PT"] > 100:

                            # how many jets (total) and total hadronic pT
                            for j in range(inputevents[i].number()["jet"]):
                                if inputevents[i]["jet"][j]["PT"] > 20 and -5.0 < inputevents[i]["jet"][j]["eta"] < 5.0:
                                    num_jets += 1
                                    HT_had += inputevents[i]["jet"][j]["PT"]


                            # photon info:
                            with open(datFolder + "BP" + str(ii) + '_photon_pruebaRO-sinMET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"]))

                            # jet info:
                            with open(datFolder + "BP" + str(ii) + '_jet_pruebaRO-sinMET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                            # lepton info
                            if inputevents[i].number()["electron"] > 0:
                                with open(datFolder + "BP" + str(ii) + '_lepton_pruebaRO-sinMET.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"]))
                            else:
                                with open(datFolder + "BP" + str(ii) + '_lepton_pruebaRO-sinMET.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"]))

                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder + "BP" + str(ii) + '_MET_pruebaRO-sinMET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                            # hadronic info
                            with open(datFolder + "BP" + str(ii) + '_hadinfo_pruebaRO-sinMET.dat', 'a') as f:
                                f.write('%f %d\n' % (HT_had, num_jets))

                            # total events that passed the cuts
                            signal_cut_ev += 1
        print('End of BP' + str(ii) + ' analysis')
        inputevents = 0 # Just to clean the memory

        print('Num initial events: ', signal_init_ev)
        print('Total pasaron los cortes: ', signal_cut_ev)

0
1


/home/martinrios/.virtualenvs/pheno2.7/lib/python2.7/site-packages/LHCO_reader/LHCO_reader.py:359: UserWarning: Couldn't read total number of events from LHCO
  warnings.warn("Couldn't read total number of events from LHCO")


Esta run tuvo 10000 eventos en el lhco
End of BP1 analysis
('Num initial events: ', 10000)
('Total pasaron los cortes: ', 2909)


## Backgrounds

### pp > tt $\gamma$ (falta NLO)

In [10]:
Folder    = '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptottbargamma/'
datFolder = '../data/ttGamma/'

ttgamma_cross = 0.079 #[pb]
ttgamma_runs  = 150
ttgamma_NLO   = 2.5 # k-factor [14 TeV] CHECK THIS
ttgamma_init_sim_eve = 100000 * ttgamma_runs

ttgamma_init_ev = 0
ttgamma_cut_ev  = 0

ttgamma_photon  = []
ttgamma_jet     = []
ttgamma_lepton  = []
ttgamma_MET     = []
ttgamma_hadinfo = []


for ii in range(1, 1 + ttgamma_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + "run100k_" + str(ii) + ".lhco")

        ttgamma_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -5.0 < inputevents[i]["jet"][0]["eta"] < 5.0:

                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.5 < inputevents[i]["photon"][0]["eta"] < 2.5:
                        if inputevents[i]["MET"][0]["PT"] > 100:

                            # how many jets (total) and total hadronic pT
                            for j in range(inputevents[i].number()["jet"]):
                                if inputevents[i]["jet"][j]["PT"] > 20 and -5.0 < inputevents[i]["jet"][j]["eta"] < 5.0:
                                    num_jets += 1
                                    HT_had += inputevents[i]["jet"][j]["PT"]


                            # photon info:
                            with open(datFolder + 'photon.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"]))

                            # jet info:
                            with open(datFolder + 'jet.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                            # lepton info
                            if inputevents[i].number()["electron"] > 0:
                                with open(datFolder + 'lepton.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"]))
                            else:
                                with open(datFolder + 'lepton.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"]))

                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder + 'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                            # hadronic info
                            with open(datFolder + 'hadinfo.dat', 'a') as f:
                                f.write('%f %d\n' % (HT_had, num_jets))

                            # total events that passed the cuts
                            ttgamma_cut_ev += 1

print('Num initial events: ', ttgamma_init_ev)
print('Total pasaron los cortes: ', ttgamma_cut_ev)

1
Esta run tuvo 55020 eventos en el lhco
2
Esta run tuvo 54714 eventos en el lhco
3
Esta run tuvo 55072 eventos en el lhco
4
Esta run tuvo 54976 eventos en el lhco
5
Esta run tuvo 54935 eventos en el lhco
6
Esta run tuvo 54733 eventos en el lhco
7
Esta run tuvo 55075 eventos en el lhco
8
Esta run tuvo 55108 eventos en el lhco
9
Esta run tuvo 55151 eventos en el lhco
10
Esta run tuvo 55887 eventos en el lhco
11
Esta run tuvo 54990 eventos en el lhco
12
Esta run tuvo 54880 eventos en el lhco
13
Esta run tuvo 55077 eventos en el lhco
14
Esta run tuvo 55014 eventos en el lhco
15
Esta run tuvo 55043 eventos en el lhco
16
Esta run tuvo 55097 eventos en el lhco
17
Esta run tuvo 54874 eventos en el lhco
18
Esta run tuvo 55182 eventos en el lhco
19
Esta run tuvo 55031 eventos en el lhco
20
Esta run tuvo 54877 eventos en el lhco
21
Esta run tuvo 54937 eventos en el lhco
22
Esta run tuvo 54992 eventos en el lhco
23
Esta run tuvo 55212 eventos en el lhco
24
Esta run tuvo 43581 eventos en el lhco
2

In [11]:
cross       = ttgamma_cross * 1000 * ttgamma_NLO
aceptancia  = ttgamma_cut_ev / (1. * ttgamma_init_sim_eve)
luminosidad = 100

ttgamma_fidcross = cross * aceptancia
ttgamma_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttgamma_fidcross)
print('Events expected: ', ttgamma_expected)

('fiducial cross section: ', 28.672233666666664)
('Events expected: ', 2867.223366666666)


### p p > W $\gamma$ > $\nu$ + l + $\gamma$ (falta chequear NLO)

In [6]:
Folder    = '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptoWgamma/'
datFolder = '../data/Wgamma/'

Wgamma_cross = 0.9102 #[pb]
Wgamma_runs  = 137
Wgamma_NLO   = 2.5 # k-factor [14 TeV] CHECK THIS
Wgamma_init_sim_eve = 100000 * Wgamma_runs

Wgamma_init_ev = 0
Wgamma_cut_ev  = 0

Wgamma_photon  = []
Wgamma_jet     = []
Wgamma_lepton  = []
Wgamma_MET     = []
Wgamma_hadinfo = []


for ii in range(1, 1 + Wgamma_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + "run100k_" + str(ii) + ".lhco")

        Wgamma_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -5.0 < inputevents[i]["jet"][0]["eta"] < 5.0:

                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.5 < inputevents[i]["photon"][0]["eta"] < 2.5:
                        if inputevents[i]["MET"][0]["PT"] > 100:

                            # how many jets (total) and total hadronic pT
                            for j in range(inputevents[i].number()["jet"]):
                                if inputevents[i]["jet"][j]["PT"] > 20 and -5.0 < inputevents[i]["jet"][j]["eta"] < 5.0:
                                    num_jets += 1
                                    HT_had += inputevents[i]["jet"][j]["PT"]


                            # photon info:
                            with open(datFolder + 'photon.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"]))

                            # jet info:
                            with open(datFolder + 'jet.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                            # lepton info
                            if inputevents[i].number()["electron"] > 0:
                                with open(datFolder + 'lepton.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"]))
                            else:
                                with open(datFolder + 'lepton.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"]))

                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder + 'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                            # hadronic info
                            with open(datFolder + 'hadinfo.dat', 'a') as f:
                                f.write('%f %d\n' % (HT_had, num_jets))

                            # total events that passed the cuts
                            Wgamma_cut_ev += 1

print('Num initial events: ', Wgamma_init_ev)
print('Total pasaron los cortes: ', Wgamma_cut_ev)

1
Esta run tuvo 96394 eventos en el lhco
2
Esta run tuvo 96333 eventos en el lhco
3
Esta run tuvo 96331 eventos en el lhco
4
Esta run tuvo 96491 eventos en el lhco
5
Esta run tuvo 96378 eventos en el lhco
6
Esta run tuvo 96527 eventos en el lhco
7
Esta run tuvo 96453 eventos en el lhco
8
Esta run tuvo 96428 eventos en el lhco
9
Esta run tuvo 96467 eventos en el lhco
10
Esta run tuvo 96433 eventos en el lhco
11
Esta run tuvo 96472 eventos en el lhco
12
Esta run tuvo 96347 eventos en el lhco
13
Esta run tuvo 96530 eventos en el lhco
14
Esta run tuvo 96510 eventos en el lhco
15
Esta run tuvo 96353 eventos en el lhco
16
Esta run tuvo 96462 eventos en el lhco
17
Esta run tuvo 96376 eventos en el lhco
18
Esta run tuvo 96341 eventos en el lhco
19
Esta run tuvo 96392 eventos en el lhco
20
Esta run tuvo 96418 eventos en el lhco
21
Esta run tuvo 96464 eventos en el lhco
22
Esta run tuvo 96371 eventos en el lhco
23
Esta run tuvo 96395 eventos en el lhco
24
Esta run tuvo 96402 eventos en el lhco
2

In [7]:
cross       = Wgamma_cross * 1000 * Wgamma_NLO
aceptancia  = Wgamma_cut_ev / (1. * Wgamma_init_sim_eve)
luminosidad = 100

Wgamma_fidcross = cross * aceptancia
Wgamma_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wgamma_fidcross)
print('Events expected: ', Wgamma_expected)

('fiducial cross section: ', 632.8869742335767)
('Events expected: ', 63288.69742335767)


### p p > W jets > $\nu$ l jets

In [ ]:
# PAPER MADGRAPH for the NLO corrections
# https://arxiv.org/pdf/1405.0301.pdf

In [2]:
Folder    = '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/'
datFolder = '../data/Wjets/'

Wjets_cross = 168.12 #[pb]
Wjets_runs  = 325
Wjets_NLO   = 1.27 # k-factor [14 TeV]
Wjets_init_sim_eve = 100000 * Wjets_runs + 2000000

Wjets_init_ev = 0
Wjets_cut_ev  = 0

Wjets_photon  = []
Wjets_jet     = []
Wjets_lepton  = []
Wjets_MET     = []
Wjets_hadinfo = []


for ii in range(1, 1 + Wjets_runs):
    print(ii)
    if ii not in [19, 33, 106, 325]: # Runs que fueron cancelados
        inputevents = LHCO_reader.Events(f_name = Folder + "run100k_" + str(ii) + ".lhco")

        Wjets_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -5.0 < inputevents[i]["jet"][0]["eta"] < 5.0:

                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.5 < inputevents[i]["photon"][0]["eta"] < 2.5:
                        if inputevents[i]["MET"][0]["PT"] > 100:

                            # how many jets (total) and total hadronic pT
                            for j in range(inputevents[i].number()["jet"]):
                                if inputevents[i]["jet"][j]["PT"] > 20 and -5.0 < inputevents[i]["jet"][j]["eta"] < 5.0:
                                    num_jets += 1
                                    HT_had += inputevents[i]["jet"][j]["PT"]


                            # photon info:
                            with open(datFolder + 'photon.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"]))

                            # jet info:
                            with open(datFolder + 'jet.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                            # lepton info
                            if inputevents[i].number()["electron"] > 0:
                                with open(datFolder + 'lepton.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"]))
                            else:
                                with open(datFolder + 'lepton.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"]))

                            # MET (eta=0 by def, but I save phi because I have many jets)
                            with open(datFolder + 'MET.dat', 'a') as f:
                                f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                            # hadronic info
                            with open(datFolder + 'hadinfo.dat', 'a') as f:
                                f.write('%f %d\n' % (HT_had, num_jets))

                            # total events that passed the cuts
                            Wjets_cut_ev += 1

print('Num initial events: ', Wjets_init_ev)
print('Total pasaron los cortes: ', Wjets_cut_ev)

1


/home/martinrios/.virtualenvs/pheno2.7/lib/python2.7/site-packages/LHCO_reader/LHCO_reader.py:359: UserWarning: Couldn't read total number of events from LHCO
  warnings.warn("Couldn't read total number of events from LHCO")


Esta run tuvo 54919 eventos en el lhco
2
Esta run tuvo 54896 eventos en el lhco
3
Esta run tuvo 54637 eventos en el lhco
4
Esta run tuvo 54863 eventos en el lhco
5
Esta run tuvo 54743 eventos en el lhco
6
Esta run tuvo 54751 eventos en el lhco
7
Esta run tuvo 54698 eventos en el lhco
8
Esta run tuvo 54635 eventos en el lhco
9
Esta run tuvo 54703 eventos en el lhco
10
Esta run tuvo 54469 eventos en el lhco
11
Esta run tuvo 54639 eventos en el lhco
12
Esta run tuvo 54783 eventos en el lhco
13
Esta run tuvo 54687 eventos en el lhco
14
Esta run tuvo 54490 eventos en el lhco
15
Esta run tuvo 54679 eventos en el lhco
16
Esta run tuvo 54319 eventos en el lhco
17
Esta run tuvo 54348 eventos en el lhco
18
Esta run tuvo 54727 eventos en el lhco
19
20
Esta run tuvo 54607 eventos en el lhco
21
Esta run tuvo 54813 eventos en el lhco
22
Esta run tuvo 54729 eventos en el lhco
23
Esta run tuvo 54470 eventos en el lhco
24
Esta run tuvo 54601 eventos en el lhco
25
Esta run tuvo 54588 eventos en el lhco


In [4]:
cross       = Wjets_cross * 1000 * Wjets_NLO
aceptancia  = Wjets_cut_ev / (1. * Wjets_init_sim_eve)
luminosidad = 100

Wjets_fidcross = cross * aceptancia
Wjets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wjets_fidcross)
print('Events expected: ', Wjets_expected)

('fiducial cross section: ', 648.9972420521739)
('Events expected: ', 64899.72420521739)
